In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Bidirectional, Input

data = {
    "Hours": [1.0, 2.5, 3.0, 4.2, 5.0, 6.0, 7.0, 8.5, 9.0],
    "Scores": [15, 25, 35, 45, 55, 65, 75, 90, 95]
}

df = pd.DataFrame(data)
print("Sample Data:\n", df.head())

X = df[['Hours']].values
y = df['Scores'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

models = {
    "Linear Regression": LinearRegression(),
    "SVM": SVR(kernel='rbf'),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(objective='reg:squarederror', random_state=42),
    "KNN": KNeighborsRegressor(n_neighbors=3),
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    results[name] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
nb = GaussianNB()
y_train_int = np.round(y_train)
nb.fit(X_train_scaled, y_train_int)
preds = nb.predict(X_test_scaled)
results["Naive Bayes (approx)"] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

mlp = MLPRegressor(hidden_layer_sizes=(64,32), max_iter=2000, random_state=42)
mlp.fit(X_train, y_train)
preds = mlp.predict(X_test)
results["MLP (Sklearn)"] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

X_train_rnn = np.expand_dims(X_train, axis=1)
X_test_rnn = np.expand_dims(X_test, axis=1)

def build_and_train(model, name):
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_rnn, y_train, epochs=200, verbose=0)
    preds = model.predict(X_test_rnn).flatten()
    results[name] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

rnn = Sequential([Input(shape=(1,1)), SimpleRNN(32), Dense(1)])
build_and_train(rnn, "RNN")

lstm = Sequential([Input(shape=(1,1)), LSTM(32), Dense(1)])
build_and_train(lstm, "LSTM")

bilstm = Sequential([Input(shape=(1,1)), Bidirectional(LSTM(32)), Dense(1)])
build_and_train(bilstm, "BiLSTM")

gru = Sequential([Input(shape=(1,1)), GRU(32), Dense(1)])
build_and_train(gru, "GRU")

bigru = Sequential([Input(shape=(1,1)), Bidirectional(GRU(32)), Dense(1)])
build_and_train(bigru, "BiGRU")

print("\nModel Performance (MSE & R2):")
print(pd.DataFrame(results).T)



Sample Data:
    Hours  Scores
0    1.0      15
1    2.5      25
2    3.0      35
3    4.2      45
4    5.0      55
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step

Model Performance (MSE & R2):
                              MSE        R2
Linear Regression       10.653734  0.989914
SVM                    935.402406  0.114412
Random Forest           20.740000  0.980364
XGBoost                162.491196  0.846162
KNN                     90.277778  0.914530
Naive Bayes (approx)    62.500000  0.940828
MLP (Sklearn)           31.226780  0.970436
RNN                   3409.972412 -2.228376
LSTM                  3695.110352 -2.498329
BiLSTM                3341.622314 -2.163666
GRU                   3478.518555 -2.293272
BiGRU                 2790.921631 -1.642293
